In [9]:
import sympy as smp
import numpy as np
import re
import itertools as it
from operator import itemgetter
from sympy.tensor.array.dense_ndim_array import MutableDenseNDimArray
m, sigma, C_0, alpha, beta, r_s, M, t, r, theta, phi, tau, x, y, z, l, E = smp.symbols('m sigma C_0 alpha beta r_s M t r theta phi tau x y z l E', float = True)

t_p, r_p, theta_p, phi_p = smp.symbols('t_p r_p theta_p phi_p', cls = smp.Function)

t_p = t_p(tau)
r_p = r_p(tau)
theta_p = theta_p(tau)
phi_p = phi_p(tau)


A , B = smp.symbols('A B', cls = smp.Function)
A = A(r)
B = B(r)

Metric = smp.MutableDenseNDimArray([[-A,0,0,0],[0,B,0,0],[0,0,r**2,0],[0,0,0,r**2*smp.sin(theta)**2]])
InvMetric = smp.MutableDenseNDimArray([[-1/A,0,0,0],[0,1/B,0,0],[0,0,1/r**2,0],[0,0,0,1/(r**2*smp.sin(theta)**2)]])
Base = smp.Array([t,r,theta,phi])
Dimention = 4
GR_General_Metric = BaseFunctions(Metric, Base)
MS = smp.Array([[-(1 - (r_s)/(r_p)),0,0,0],[0,1/(1 - (r_s)/(r_p)),0,0],[0,0,r_p**2,0],[0,0,0,r_p**2*smp.sin(theta_p)**2]])

Ricci = GR_General_Metric.Ricci()
RimannCov = GR_General_Metric.Riemann0000()

In [10]:
def conditianal(x):
    if isinstance(x, int):
        return x
    else:
        return x.subs(r, phi)
def deff(x):
    return x if isinstance(x, int) else x.subs(r, phi)

In [11]:
single_transformation = 'x = y'
coordinate_transformation_string = '[x = r*cos(theta), y = r*sin(theta)]'

obj = re.split('[=]', single_transformation.replace(' ', ''))
obj

['x', 'y']

In [12]:
InvMetric.applyfunc(lambda x: x if (isinstance(x, int) or isinstance(x, float)) else x.subs(r, phi))

[[-1/A(phi), 0, 0, 0], [0, 1/B(phi), 0, 0], [0, 0, phi**(-2), 0], [0, 0, 0, 1/(phi**2*sin(theta)**2)]]

In [13]:
class BaseFunctions:
    def __init__(self, Metric, Basis):
        self.Metric = Metric
        self.Basis = Basis
        self.Dimention = len(Basis)
        
        
    def Derivative(self):
        N = self.Dimention
        A = smp.MutableDenseNDimArray(smp.zeros(N**3),(N,N,N))
        for i in range(N):
            for j in range(N):
                for k in range(N):
                    A[i,j,k] = smp.diff(self.Metric[j,k],self.Basis[i])
        return smp.simplify(A)
    
    
    def Ginv(self):
        N = self.Dimention
        g_m = self.Metric.tomatrix()
        inv_g = g_m.inv()
        A = smp.MutableDenseNDimArray(smp.zeros(N**2),(N,N))
        for i in range(N):
            for j in range(N):
                A[i,j] = inv_g[i, j]
        return smp.simplify(A)
    
    
    def Gamma(self):
        N = self.Dimention
        A = smp.MutableDenseNDimArray(smp.zeros(N**3),(N,N,N))
        ig = self.Ginv()
        for i in range(N):
            for j in range(N):
                for k in range(N):
                    for d in range(N):
                        A[i, j, k] += smp.Rational(1, 2)*(ig[d,i])*(smp.diff(self.Metric[k,d],self.Basis[j]) + smp.diff(self.Metric[d,j],self.Basis[k]) - smp.diff(self.Metric[j,k],self.Basis[d]))
        return smp.simplify(A)
    
    
    def TDerivative(self, V):
        N = self.Dimention
        A = smp.MutableDenseNDimArray(smp.zeros(N**3),(N,N,N))
        for i in range(N):
            for j in range(N):
                for k in range(N):
                    A[i,j,k] = smp.diff(V[j,k],self.Basis[i])
        return smp.simplify(A)
    
    def CovariantD10(self, V):
        N = self.Dimention
        C = self.Gamma()
        A = smp.MutableDenseNDimArray(smp.zeros(N**2),(N,N))
        for i in range(N):
            for j in range(N):
                for k in range(N):
                    A[i,j] += smp.Rational(1, N)*smp.diff(V[j],self.Basis[i]) + C[j,i,k]*V[k]
        return smp.simplify(A)
    
    
    def CovariantD01(self, V):
        N = self.Dimention
        C = self.Gamma()
        A = smp.MutableDenseNDimArray(smp.zeros(N**2),(N,N))
        for i in range(N):
            for j in range(N):
                for k in range(N):
                    A[i,j] += smp.Rational(1, N)*smp.diff(V[j],self.Basis[i]) - C[k,i,j]*V[k]
        return smp.simplify(A)
    
    
    def CovariantD20(self, T):
        N = self.Dimention
        C = self.Gamma()
        A = smp.MutableDenseNDimArray(smp.zeros(N**3),(N,N,N))
        for i in range(N):
            for j in range(N):
                for k in range(N):
                    for p in range(N):
                        A[i,j,k] += smp.Rational(1, N)*smp.diff(T[j,k],self.Basis[i]) + C[j,i,p]*T[p,k] + C[k,i,p]*T[p,j]
        return smp.simplify(A)
    
    
    def CovariantD02(self, T):
        N = self.Dimention
        C = self.Gamma()
        A = smp.MutableDenseNDimArray(smp.zeros(N**3),(N,N,N))
        for i in range(N):
            for j in range(N):
                for k in range(N):
                    for p in range(N):
                        A[i,j,k] += smp.Rational(1, N)*smp.diff(T[j,k],self.Basis[i]) - C[p,i,j]*T[p,k] - C[p,i,k]*T[p,j]
        return smp.simplify(A)
    
    
    def CovariantD11(self, T):
        N = self.Dimention
        C = self.Gamma()
        A = smp.MutableDenseNDimArray(smp.zeros(N**3),(N,N,N))
        for i in range(N):
            for j in range(N):
                for k in range(N):
                    for p in range(N):
                        A[i,j,k] += smp.Rational(1, N)*smp.diff(T[j,k],self.Basis[i]) + C[j,i,p]*T[p,k] - C[p,i,k]*T[j,p]
        return smp.simplify(A)
    
    
    def Riemann1000(self):
        N = self.Dimention
        C = self.Gamma()
        A = smp.MutableDenseNDimArray(smp.zeros(N**4),(N,N,N,N))
        for i in range(N):
            for j in range(N):
                for k in range(N):
                    for p in range(N):
                        for d in range(N):
                            A[i, j, k, p] += smp.Rational(1, N)*(smp.diff(C[i,p,j],self.Basis[k])-
                                                             smp.diff(C[i,k,j],self.Basis[p]))+(C[i,k,d]*C[d,p,j]-C[i,p,d]*C[d,k,j])
        return smp.simplify(A)
    
    def Riemann0000(self):
        N = self.Dimention
        R = self.Riemann1000()
        A = smp.MutableDenseNDimArray(smp.zeros(N**4),(N,N,N,N))
        for i in range(N):
            for j in range(N):
                for k in range(N):
                    for p in range(N):
                        for d in range(N):
                            A[i,j,k,p] += self.Metric[i,d]*R[d,j,k,p]
        return smp.simplify(A)
    
    def Riemann0100(self):
        N = self.Dimention
        R = self.Riemann1000()
        A = smp.MutableDenseNDimArray(smp.zeros(N**4),(N,N,N,N))
        for i in range(N):
            for j in range(N):
                for k in range(N):
                    for p in range(N):
                        for d in range(N):
                            A[i,j,k,p] += self.Metric[i,d]*R[d,j,k,p]
        return smp.simplify(A)
    
    def Ricci(self):
        N = self.Dimention
        ig = self.Ginv()
        CR = self.Riemann0000()
        A = smp.MutableDenseNDimArray(smp.zeros(N**2),(N,N))
        for i in range(N):
            for j in range(N):
                for d in range(N):
                    for s in range(N):
                        A[i,j] += ig[d,s]*CR[d,i,s,j]
        return smp.simplify(A)
    
    
    def ricscalar(self):
        N = self.Dimention
        R = self.Ricci()
        ig = self.Ginv()
        A = float()
        for d in range(N):
            for s in range(N):
                A += ig[d,s]*R[d,s]
        return smp.simplify(A)
    

    def kscalar(self):
        N = self.Dimention
        R = self.Riemann0000()
        ig = self.Ginv()
        A = float()
        for i in range(N):
            for j in range(N):
                for k in range(N):
                    for p in range(N):
                        for d in range(N):
                            for n in range(N):
                                for s in range(N):
                                    for t in range(N):
                                        A += ig[i,j]*ig[k,p]*ig[d,n]*ig[s,t]*R[i,k,d,s]*R[j,p,d,s]
        return smp.simplify(A)
    
    
    def Geodesic(self,t):
        N = self.Dimention
        C = self.Gamma()
        A = smp.MutableDenseNDimArray(smp.zeros(N),(N))
        for i in range(N):
            for j in range(N):
                for k in range(N):
                    A[[i]] += smp.Rational(1, N**2)*(smp.diff(self.Basis[i], t, t)) + C[i, j, k]*(smp.diff(self.Basis[j], t))*(smp.diff(self.Basis[k], t))
        return smp.simplify(A)
    
    
    def Lagrangian(self, t):
        N = self.Dimention
        A = float()
        for i in range(N):
            for j in range(N):
                A += - self.Metric[i,j]*smp.diff(self.Basis[i], t)*smp.diff(self.Basis[j], t)
    
        return smp.sqrt(A)
    
    
    def EulerLagrange(self,t):
        L = self.Lagrangian(t)
        N = self.Dimention
        A = smp.MutableDenseNDimArray(smp.zeros(N),(N))
        for i in range(N):
            A[[i]] = smp.diff(self.Basis[i],t)
        
        
        B = smp.MutableDenseNDimArray(smp.zeros(N),(N))
        C = smp.MutableDenseNDimArray(smp.zeros(N),(N))
        for i in range(N):
            C[[i]] = smp.diff(L, A[i])
        for i in range(N):
            B[[i]] = C[i].subs(L, 1)
    
    
        D = smp.MutableDenseNDimArray(smp.zeros(N),(N))
        E = smp.MutableDenseNDimArray(smp.zeros(N),(N))
        for i in range(N):
            D[[i]] = smp.diff(L, self.Basis[i])
        for i in range(N):
            E[[i]] = D[i].subs(L, 1)
    
        F = smp.MutableDenseNDimArray(smp.zeros(N),(N))
        for i in range(N):
            F[[i]] = E[i] - smp.diff(B[i],t)
        
        return F
       
    def Einstein(self, StressEnergy, Cosmological, SI_Units):
        G = self.Metric
        T = StressEnergy
        N = self.Dimention
        Lambda  = smp.symbols('Lambda')
        Ric = self.Ricci()
        RScalar = self.RicciScalar()
        E = smp.MutableDenseNDimArray(smp.zeros(N**2),(N,N))
        
        if Cosmological and not SI_Units: 
            for i in range(N):
                for j in range(N):
                    E[i,j] = Ric[i,j] - smp.Rational(1,2)*G[i,j]*RScalar + (Lambda)*G[i,j] - (8*smp.pi)*T[i,j]
            return E
           
        if Cosmological and SI_Units: 
            for i in range(N):
                for j in range(N):
                    E[i,j] = Ric[i,j] - smp.Rational(1,2)*G[i,j]*RScalar + (Lambda)*G[i,j] - (8*smp.pi)*T[i,j]
            return E
           
        if not Cosmological and not SI_Units: 
            for i in range(N):
                for j in range(N):
                    E[i,j] = Ric[i,j] - smp.Rational(1,2)*G[i,j]*RScalar - (8*smp.pi)*T[i,j]
            return E
           
        if not Cosmological and SI_Units: 
            for i in range(N):
                for j in range(N):
                    E[i,j] = Ric[i,j] - smp.Rational(1,2)*G[i,j]*RScalar - (8*smp.pi)*T[i,j]
            return E

In [14]:
import re
import sympy as smp
import numpy as np
from sympy import sin, cos, tan, sinh, cosh, tanh, exp, asin, acos, atan

class StringParser:
    def __init__(self, StringExpression):
        self.StringExpression = StringExpression
    
    def Functions(self):
        Function = "(?<![a-zA-Z])[a-zA-Z](?=\()"
        RemoveSpaces = self.StringExpression.replace(' ', '')
        FunctionList = []
        List = []
        FunctionList = [x for x in re.finditer(Function, RemoveSpaces)]
        for i in FunctionList:
            List.append(i.group())
        return list(dict.fromkeys(List))
            
    
    def symbolRecognizer(self, Input):
        GreekSymbols = "(theta|phi|omega|sigma|alpha|beta|gamma|epsilon|zeta|eta|kappa|lambda|mu|nu|pi|Theta|Phi|Omega|Sigma|Alpha|Beta|Gamma|Epsilon|Zeta|Eta|Kappa|Lambda|Mu|Nu|Pi)"
        RemoveSpaces = Input.replace(' ', '')
        ListOne = []
        ListOne = [x for x in re.finditer("(?<![a-zA-Z])[a-zA-Z](?![a-zA-Z])", RemoveSpaces)]
        ListTwo = [x for x in re.finditer(GreekSymbols, RemoveSpaces)]

        for i in ListTwo:
            ListOne.append(i)
        return ListOne

    def preDefinedFunctionRecognizer(self):
        SpecialFunctions = ['(sin)', '(cos)', '(tan)', '(sinh)', '(cosh)', '(tanh)', '(asin)', '(atan)', '(acos)', '(asinh)', '(acosh)', '(atanh)', '(exp)', '(pi)']
        RemoveSpaces = self.StringExpression.replace(' ', '')
        ListOfFunctions = []
        for i in SpecialFunctions:
            if [x for x in re.finditer(i, RemoveSpaces)] != []:
                ListOfFunctions.append([x for x in re.finditer(i, RemoveSpaces)])
        return ListOfFunctions

    #symbolRecognizer(Input)[1].group() = "F"
    #symbolRecognizer(Input)[1].span() = (17,18)

    def replacer(self, Input, Replace, Location):
        NewString = Input.replace(' ', '')
        return NewString[:Location[0]] + Replace + NewString[Location[1]:]

    #replacer(Input, 'Q[1]', symbolRecognizer(Input)[1].span())

    def returnLists(self):
        AllSymbolsList = []
        VariableList = []
        FunctionList = []
        
        for i in range(len(self.symbolRecognizer(self.StringExpression))):
            AllSymbolsList.append(self.symbolRecognizer(self.StringExpression)[i].group())
        NoDuplicatesAllSymbolsList = list(dict.fromkeys(AllSymbolsList))
        
        for i in NoDuplicatesAllSymbolsList:
            if i not in self.Functions():
                VariableList.append(i)
            else:
                FunctionList.append(i)

        return [VariableList,FunctionList,AllSymbolsList]

    def returnDictionary(self):
        VariableList = self.returnLists()[0]
        FunctionList = self.returnLists()[1]
        Dic = {}
        if len(VariableList) == 1 and len(FunctionList) == 1:
            for i in range(len(VariableList)):
                Dic.update({VariableList[i] : 'W'.format(i)})
            for i in range(len(FunctionList)):
                Dic.update({FunctionList[i] : 'Q'.format(i)})
        elif len(VariableList) == 1 and len(FunctionList) == 0:
            for i in range(len(VariableList)):
                Dic.update({VariableList[i] : 'W'.format(i)})
        elif len(VariableList) > 1 and len(FunctionList) == 0:
            for i in range(len(VariableList)):
                Dic.update({VariableList[i] : 'W[{}]'.format(i)})
        elif len(VariableList) > 1 and len(FunctionList) > 1:
            for i in range(len(VariableList)):
                Dic.update({VariableList[i] : 'W[{}]'.format(i)})
            for i in range(len(FunctionList)):
                Dic.update({FunctionList[i] : 'Q[{}]'.format(i)})
        elif len(VariableList) == 1 and len(FunctionList) > 1:
            for i in range(len(VariableList)):
                Dic.update({VariableList[i] : 'W'.format(i)})
            for i in range(len(FunctionList)):
                Dic.update({FunctionList[i] : 'Q[{}]'.format(i)})
        elif len(FunctionList) > 1 and len(VariableList) == 1:
            for i in range(len(FunctionList)):
                Dic.update({FunctionList[i] : 'Q[{}]'.format(i)})
            for i in range(len(FunctionList)):
                Dic.update({FunctionList[i] : 'Q'.format(i)})
        elif len(FunctionList) == 1 and len(VariableList) > 1:
            for i in range(len(FunctionList)):
                Dic.update({FunctionList[i] : 'Q'.format(i)})
            for i in range(len(VariableList)):
                Dic.update({VariableList[i] : 'W[{}]'.format(i)})
        return Dic

    def returnSympyString(self):
        New = self.StringExpression
        for i in range(len(self.symbolRecognizer(self.StringExpression))):
            New = self.replacer(New,self.returnDictionary()[self.symbolRecognizer(New)[i].group()], self.symbolRecognizer(New)[i].span())
        return New



class SympyParser:
    def __init__(self, equationString):
        self.equationString = equationString

    # This will convert a list of variables and convert them into one string, in the same order of the List, 
    # so that sympy can parse the string to sympy objects.
    # Input: List = ['x', 'y', 'z']
    # Output: String = 'x y z '
    def convertListToSympyVariableString(self, stringList):
        varableList = []
        for string in stringList:
            varableList.append(string.replace(' ',''))
        variable = ''
        for i in range(len(varableList)):
            variable += varableList[i] + ' '
        return variable
   
    def convertToSympyObject(self):
        VariableSymbolsList = StringParser(self.equationString).returnLists()[0]
        FunctionSymbolList = StringParser(self.equationString).returnLists()[1]
        FunctionSymbols = self.convertListToSympyVariableString(FunctionSymbolList)
        VariableSymbols = self.convertListToSympyVariableString(VariableSymbolsList)
        
        if int(len(StringParser(self.equationString).returnLists()[0])) == 0:
            return smp.symbols(StringParser(self.equationString).returnSympyString())
            #return eval(StringParser(self.equationString, self.functionList).returnSympyString())
        elif int(len(StringParser(self.equationString).returnLists()[1])) == 0:
            W = smp.symbols(VariableSymbols)
            return eval(StringParser(self.equationString).returnSympyString())
        elif int(len(StringParser(self.equationString).returnLists()[1])) >= 1:
            W = smp.symbols(VariableSymbols)
            Q = smp.symbols(FunctionSymbols, cls = smp.Function)
            return eval(StringParser(self.equationString).returnSympyString())

In [15]:
class Index:
    def __init__(self, index_symbol, order, running_index, component_type, basis, index_value = None):
        self.index_symbol = index_symbol
        self.order = order
        self.running_index = running_index
        self.component_type = component_type
        if isinstance(basis, str):
            self.basis = SympyParser(basis).convertToSympyObject()
        else:
            self.basis = basis
        self.dimention = int(len(self.basis))
        if not self.running_index and isinstance(index_value, int):
            self.index_value = index_value
        elif self.running_index:
            self.index_value = [i for i in range(self.dimention)]
        else:
            raise ValueError("If the index is not a running index (running_index=False) then the index_value argument must be specified as an integer.")
        self._ = slice(0, self.dimention) if self.running_index else slice(self.index_value, self.index_value+1)
        
    def _is_contravariant(self):
        return self.component_type == 'contravariant'
    
    def _is_covariant(self):
        return self.component_type == 'covariant'
    
    def __eq__(self, other):
        if self.index_symbol == other.index_symbol and self.running_index == other.running_index and self.component_type == other.component_type:
            return True
        else:
            return False

    def __neg__(self):
        if self.component_type == 'covariant':
            return Index(self.index_symbol, self.order, self.running_index, 'contravariant', self.basis, self.index_value)
        else:
            return Index(self.index_symbol, self.order, self.running_index, 'covariant', self.basis, self.index_value)

    def __repr__(self):
        """
        Explain method.
        """
        return f"""Index Object:( 
                    'index symbol' : {self.index_symbol}, \n\
                    'order' : {self.order}, \n\
                    'dimention' : {self.dimention}, \n\
                    'running index' : {self.running_index}, \n\
                    'index value(s)' : {self.index_value}, \n\
                    'component type' : {self.component_type} \n\
                )
                """

    def set_start(self):
        """
        Future note: We could easily implement slices feature for indices. So users can see a slice of a tensor specified via the indices.
        Simply add running indices start of slice and end on end of slice.
        """
        if self.running_index:
            return 0
        elif not self.running_index and isinstance(self.index_value, int):
            return self.index_value

    def set_end(self):
        if self.running_index:
            return self.dimention - 1
        elif not self.running_index and isinstance(self.index_value, int):
            return self.index_value
 
    def __iter__(self):
        self.first_index_value = self.set_start()
        self.last_index_value = self.set_end()
        return self

    def __next__(self):
        if self.first_index_value <= self.last_index_value:
            x = self.first_index_value
            self.first_index_value += 1
            return x
        else:
            raise StopIteration

    def __index__(self):
        if not self.running_index:
            return self.constant_value
        else:
            raise ValueError("The index {} you have entered has not been assigned a constant.".format(self.index_symbol))
        
    def __str__(self):
        return self.__repr__()

    def __len__(self):
        return self.dimention
    
    def __mul__(self, other):
        """
        ---- Einstein Summation Convention Inplementation ------
        
        This is a personal definition, for simplicity within the rest of the application.
        When two tensors are multiplied together and the einstein summation convetion is true, this will return the location order of those indices.
        
            Tensor_{a}_{b} * Tensor^{b}_{c} = Tensor_{a}_{c}
        
        This operation will return the following when we aply thit logic to the combinatorials of these indices:
        
            [ [1, 0] ] => Which is saying that the index at location/order 1 of the first tensor is being summed with index at location/order 0 from the second tensor.
        """
        if isinstance(other, Index):
            if self.index_symbol == other.index_symbol and self.component_type != other.component_type and id(self) != id(other):
                return [self.order, other.order]
            else:
                return []
        else:
            raise ValueError("Cannot perform multiplication for the objects you have entered as {}".format(type(other)))
            
    def __add__(self, other):
        """
        ---- Summation of Tesnors ------
        
        This is a personal definition, for simplicity within the rest of the application and simliar but different to the multiplication one.
        When two tensors are added or subtracted together together, the resulting expression is ONLY a tensor expression if the two tensors being added/subtracted have the same index structure.
        However, the indices themselves can sometimes be in different orders:
        
            Tensor_{a}_{b} + Tensor_{b}_{a} = Tensor_{a}_{b}
            Tensor_{a}_{b} - Tensor_{b}_{a} = Tensor_{a}_{b}
            Tensor_{a}_{b} - Tensor_{a}_{b} = Tensor_{a}_{b}
            
        ---- Example One ----
        
            Tensor_{a}_{b} + Tensor_{b}_{a} = Tensor{_{a}_{b}}:
            [ [1, 0],[0,1] ] => Which is saying that the index at location/order 1 of the first tensor is the same as the index at location/order 0 from the second tensor.
                                and index at location/order 0 of the first tensor is the same as the index at location/order 1 from the second tensor.
                                
        ---- Example Two ----
        
            Tensor_{a}_{b} + Tensor_{a}_{b} = Tensor_{a}_{b}:
            [ [0, 0],[1,1] ] => Which is saying that the index at location/order 0 of the first tensor is the same as the index at location/order 0 from the second tensor.
                                and index at location/order 1 of the first tensor is the same as the index at location/order 1 from the second tensor.
        """
        if isinstance(other, Index):
            if self.index_symbol == other.index_symbol and self.component_type == other.component_type and id(self) != id(other):
                return [self.order, other.order]
            else:
                return []
        else:
            raise ValueError("Cannot perform addition for the objects you have entered as {}".format(type(other)))
            
    def __index__(self):
        if not self.running_index:
            return self.index_value
        else:
            raise ValueError("The index {} is a running index and has not been assigned a number. Index class has the slice property as Index._ which you may use to return a slice of the object.".format(self.index_symbol))


In [16]:
class Tensor:
    """
    Tensor Class
    
    This class can be used to define tensors, but they are also the Base classes for the GrTensor class.
    """
    
    def __init__(self, components, basis):
        """
        Constructor

        Parameters
        ----------
        components : str OR MutableDenseNDimArray
            If componets of type str, then we generate a MutableDenseNDimArray from the string using SympyParser.
            If componets of type MutableDenseNDimArray, then we pass directly to the class property.
        basis : str OR MutableDenseNDimArray
            If componets of type str, then we generate a MutableDenseNDimArray from the string using SympyParser.
            If componets of type MutableDenseNDimArray, then we pass directly to the class property.
            
        """
        if isinstance(components, str):
            self.components = SympyParser(components).convertToSympyObject()
        elif isinstance(components, MutableDenseNDimArray):
            self.components = components
        else:
            raise ValueError("Component input must be either a string or a sympy object of type MutableDenseNDimArray.")
    
        self.dimention = None
        #else:
        #    raise ValueError("Basis input must be either a string or a sympy object of type MutableDenseNDimArray.")

    def __str__(self):
        """
        Explain method.
        """
        return str(self.components)
    
    def __repr__(self):
        """
        Explain method.
        """
        return f"""Tensor Object:( \n\
        Components : {self.components}
        )
        """
    
    def __mul__(self, other):
        """
        Multiplication
        """
        return smp.tensorproduct(self.components, other.components)
    
    def __rmul__(self, other):
        return self.__mul__(other)

# Index Classclass Counter:np.arrange(0,4)

For a Workflow:

Store any defined tensor in json format, with key value pair. Where the key represents the name you have named you tensor object.
Once a user passes in a string, it will check the stored keys and see if there is a tensor named the name the user has entered.
If there is one, then the calculator will calculate the expression with the values stored for these keys.

In [17]:
class GrIndex(Index):
    """
    THIS WILL BECOME A STRING PARSER FOR SPECIFIC REPRESENTATION OF A STRING.
    CHANGE NAME TO SOMETHING SPECIFIC TO THIS PARSER.
    Class Representing an index in General Relativity, encoded as covariant or contravariant and also summation convation.
    
    Here are a few examples of how the data structure of the class can look like:
    
    This class decodes the string representation of the form: _{ index_symbol (= index_value) }
    
        Ex1 = {
                    'index string representation' : '_{a}',
                    'order' : 0,
                    'index symbol' : 'a',
                    'running index' : True,
                    'index slice' : slice(0, dimention),
                    'index space' : 'cotangent',
                    'component type' : 'covariant',
                    'dimention' : 4
                 }

        Ex2 = {
                    'index string representation' : '^{b=2}',
                    'order' : 1,
                    'index symbol' : 'b',
                    'running index' : False,
                    'index slice' : slice(2,3),
                    'index space' : 'tangent',
                    'component type' : 'contravariant',
                    'dimention' : 4
                 }
                 
    Some ideas to extend this class are:
        - Notice how everything is generic, but the string representation. In other words, the representation encodes most of the properties of the class.
          So, inprinciple we can simple define other representations which encode the same information, say for instance Latex index format.
          The class should be written such that as long as everything within the class carries from the representation and dimention, then the whole package
          should still work. I.e. the class should be extensible to other index string representations.
    
    Note:   
    Covariant => Lower indices
    Contravariant => Upper indices

    """
    integers = {'0': 0, '1':1, '2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9}
    def __init__(self, index_representation, order, coordinate_basis):
        """
        Explain Constructor parameters
        This class inherits the Index class:
            Index(index_symbol, order, running_index, component_type, basis, index_value = None)
            
        """
        integers = {'0': 0, '1':1, '2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9}
        self.index_string_representation = self.__str_rep_is_valid(index_representation)
        Index.__init__(self, 
                       index_symbol = re.search('[a-zA-Z]+', self.index_string_representation).group(),
                       order = self.__is_valid_order(order),
                       running_index = not bool(re.search('^[^=]*(=)([0-9]+)[^=]*$', self.index_string_representation)),
                       component_type = 'covariant' if (self.index_string_representation[0] == "_") else 'contravariant',
                       basis = self.__is_valid_basis(coordinate_basis),
                       index_value = integers[re.split('[=]', self.index_string_representation.replace('{','').replace('}','').replace('_','').replace('^',''))[1].replace(" ","")] if bool(re.match('^[^=]*(=)([0-9]+)[^=]*$', self.index_string_representation)) else None
                       )

        
    def __is_valid_basis(self, basis):
        return basis
    
    def __str_rep_is_valid(self, index):
        index_ = index.replace(" ", "")
        condition1 = index_[0] == "_" or index_[0] == "^"
        condition2 = index_[1] == "{"
        condition3 = index_[-1] == "}"
        condition4 = bool(re.search('^[^=]*(=)([0-9]+)[^=]*$', index_)) or bool(re.search('^[^=]*[a-zA-Z]+[^=]*$', index_))
        if not condition1:
            raise ValueError("The index {} you have entered, does not contain the necessary characters _ OR ^ to indicate covariace or contravariance.".format(index_))
        elif not condition2 or not condition3:
            raise ValueError("The index {} you have entered, does not contain closing OR opening curly brakets.".format(index_))
        elif not condition4:
            raise ValueError("The following characters you have entered in the index are not recognized: {}".format(index_.replace('{','').replace('}','').replace('_','').replace('^','')))
        else:
            return index_

    def __is_valid_order(self, order):
        valid = type(order) == int
        if valid:
            return order
        else:
            raise ValueError("Index order must be of type: int")
        

    def __eq__(self, other):
        if self.index_string_representation == other.index_string_representation:
            return True
        else:
            return False

    def __neg__(self):
        if self.index_string_representation[0] == '_':
            return GrIndex(self.index_string_representation.replace('_', '^'), self.order, self.basis)
        else:
            return GrIndex(self.index_string_representation.replace('^', '_'), self.order, self.basis)

    def __repr__(self):
        """
        Explain method.
        """
        return f"""GrIndex Object:( 
                    'index string representation' : {self.index_string_representation}, \n\
                    'index symbol' : {self.index_symbol}, \n\
                    'order' : {self.order}, \n\
                    'dimention' : {self.dimention}, \n\
                    'running index' : {self.running_index}, \n\
                    'index value(s)' : {self.index_value}, \n\
                    'component type' : {self.component_type} \n\
                )
                """

    def __index__(self):
        if not self.running_index:
            str_num = re.split('[=]', self.index_string_representation.replace('{','').replace('}','').replace('_','').replace('^',''))[1]
            if str_num in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']:
                return int(eval(str_num))
            else:
                raise ValueError("The index {} you have entered, is not an integer number.".format(self.index_string_representation))
        else:
            raise ValueError("The index {} you have entered has not been assigned a constant.".format(self.index_string_representation))


Goal:

Indices String -> String Categorization Object -> string passed into a relevent object which inherits the Base Indices obeject.

In [18]:
class IndexCategorizer:
    """
    IMPORTANT: Always try and break this class by checking whether a string passes the match of more than one Category!!
    Optional Use class.
    """
    def __init__(self, indices, basis):
        if isinstance(indices, str):
            self.indices_string = indices
        elif isinstance(indices, Indices) or isinstance(indices, GrIndices):
            self.indices_string = ''
            self.indices_object = indices
        else:
            raise ValueError("The IndexCategorizer only takes strings as arguments, please make sure the object you are passing into this object is a string.")
        if isinstance(basis, str):
            self.basis = SympyParser(basis).convertToSympyObject()
        else:
            self.basis = basis
        if self.is_category_one():
            self.indices_object = GrIndices(self.indices_string, self.basis)
        
    def is_category_one(self):
        """
        Example of string to match this category:
            ^{a}^{b}_{theta = 0}_{phi=1}
        
        Conditions for this category to be recognized: 
            - Cannot contain anything but: = or { or } or _ or ^ or [a-zAZ0-9]
            - Has correct pattern: _{}^{}...
            - Between every curly brackets {}, there must be at least one [A-Za-z]+ character/word.
        """
        return bool(re.search("^((\^|\_)(\{)(\}))+$", re.sub('[^\^^\_^\{^\}]',"", self.indices_string).replace(" ",'')))

In [22]:
IndexCategorizer("^{a}^{b}_{theta=0}_{phi=1}", Base).is_category_one()

NameError: name 'GrIndices' is not defined

In [23]:
index_symbols_list = ["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z",
                      "theta","phi","alpha","beta","mu","nu","sigma","delta","omega","gamma","alpha","beta","epsilon","zeta",
                      "eta","kappa","lambda","pi"]
ind = ["a","b","c","d","e"]

list(set(index_symbols_list) - set(ind))[0]

def Array(Dimention, Rank):
    return list(it.product(np.arange(Dimention), repeat = Rank))

Array(4, 3)

[(0, 0, 0),
 (0, 0, 1),
 (0, 0, 2),
 (0, 0, 3),
 (0, 1, 0),
 (0, 1, 1),
 (0, 1, 2),
 (0, 1, 3),
 (0, 2, 0),
 (0, 2, 1),
 (0, 2, 2),
 (0, 2, 3),
 (0, 3, 0),
 (0, 3, 1),
 (0, 3, 2),
 (0, 3, 3),
 (1, 0, 0),
 (1, 0, 1),
 (1, 0, 2),
 (1, 0, 3),
 (1, 1, 0),
 (1, 1, 1),
 (1, 1, 2),
 (1, 1, 3),
 (1, 2, 0),
 (1, 2, 1),
 (1, 2, 2),
 (1, 2, 3),
 (1, 3, 0),
 (1, 3, 1),
 (1, 3, 2),
 (1, 3, 3),
 (2, 0, 0),
 (2, 0, 1),
 (2, 0, 2),
 (2, 0, 3),
 (2, 1, 0),
 (2, 1, 1),
 (2, 1, 2),
 (2, 1, 3),
 (2, 2, 0),
 (2, 2, 1),
 (2, 2, 2),
 (2, 2, 3),
 (2, 3, 0),
 (2, 3, 1),
 (2, 3, 2),
 (2, 3, 3),
 (3, 0, 0),
 (3, 0, 1),
 (3, 0, 2),
 (3, 0, 3),
 (3, 1, 0),
 (3, 1, 1),
 (3, 1, 2),
 (3, 1, 3),
 (3, 2, 0),
 (3, 2, 1),
 (3, 2, 2),
 (3, 2, 3),
 (3, 3, 0),
 (3, 3, 1),
 (3, 3, 2),
 (3, 3, 3)]

In [24]:
class Indices:
    
    """
    Note before alteration of this class:
    
    Do not create a dependence between this class and any of the deserializing classes.
    
    The responsibility of the serializing classes is to decompose the information from strings,
    of different formats. Then to pass those information into this Indices class.
    
    -------- EXMAPLE --------
    
    Index Object:( 
                    'index symbol' : a, 
                    'Coordinate basis' : [t, r, theta, phi],
                    'order' : 0, 
                    'dimention' : 4, 
                    'running index' : True, 
                    'index value(s)' : [0, 1, 2, 3], 
                    'component type' : covariant 
                )

    Index Object:( 
                    'index symbol' : b, 
                    'Coordinate basis' : [t, r, theta, phi],
                    'order' : 1, 
                    'dimention' : 4, 
                    'running index' : True, 
                    'index value(s)' : [0, 1, 2, 3], 
                    'component type' : covariant 
                )
                
    Resulting Indices Object:

    Indices Object:(
                'index objects' : [Index, Index],
                'Coordinate basis' : [t, r, theta, phi],
                'is valid index structure' : True,
                'dimention' : 4,
                'rank' : (0,2),
                'tensor components shape' : (4, 4),
                'is a scalar' : False,
                'summed indices' : []
            )
    """
    index_symbols_list = ["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z",
                          "theta","phi","alpha","beta","mu","nu","sigma","delta","omega","gamma","alpha","beta","epsilon","zeta",
                          "eta","kappa","lambda","pi"]

    def __init__(self, index_objects, basis):
        self.index_objects = index_objects
        if isinstance(basis, str):
            self.basis = SympyParser(basis).convertToSympyObject()
        else:
            self.basis = basis
        self.is_valid_index_structure = self.__validate_index_structure()
        self.dimention = int(len(self.basis))
        self.rank_as_tuple = self.return_rank_structure_as_tuple()
        self.is_scalar = len(self.index_objects) == 0
        self.tensor_component_shape = tuple([i.dimention for i in self.index_objects])
        self._ = tuple([x._ for x in self.index_objects])
        self.__indices_iterator = list(it.product(*[x for x in self.index_objects]))

    def __validate_index_structure(self):
        """
            Does the index's orders match the order to which they are within the list? -> No? -> raise Error
            Are all the index's dimentions the same? -> No? raise Error
            Are all the index's basis the same? -> No? raise Error
            Are all the index's symbols different to one another |-> No? -> is one a contravariant and another a covariant? |-> No? raise Error
                                                                 |-> Yes? -> do nothing
                                                                                                                            |-> Yes? add to summed indices
        """
        return True
    
    def __iter__(self):
        self.__length = int(len(self.__indices_iterator))
        self.__i = 1
        return self

    def __next__(self):
        """
        Method called when object is passed thorugh an iteration (for or while loops)
        For our case, we simply iterate through the combinations of all the combinatorials.
        These combinatorials are reflective of 
        """
        if self.__i <= self.__length:
            x = self.__i
            self.__i += 1
            return self.__indices_iterator[x-1]
        else:
            raise StopIteration
            
    def __repr__(self):
        """
        Explain method.
        """
        return f"""Indices Object:( 
                    'index objects' : {self.index_objects}, \n\
                    'Coordinate basis' : {self.basis}, \n\
                    'is valid index structure' : {self.is_valid_index_structure}, \n\
                    'dimention' : {self.dimention}, \n\
                    'rank' : {self.rank_as_tuple}, \n\
                    'tensor components shape' : {self.tensor_component_shape}, \n\
                    'is a scalar' : {self.is_scalar} \n\
                )
                """
    def __eq__(self, other_indices):
        """
        When both indices have equal symbols and equal number of covarient and/or contravariant indices in any order:
        ---- TRUE example ---
        _{a}_{b} == _{b}_{a} -> true (We can letter use this as a property to define commutator of indices)
        _{a}_{b} == _{a}_{b} -> true
        ^{a}_{b} == _{b}^{a} -> true
        
        ---- FALSE example ---
        _{a}^{b} == _{b}^{a} -> false
        _{a}_{b} == ^{b}_{a} -> false
        _{a}_{b} == _{a}_{b}_{c} -> false
        _{a}_{b} == _{a}_{c} -> false
        
        """
        boolean_combinatorial_list = [i==j for (i, j) in list(it.product(self.index_objects, other_indices.index_objects))]
        return boolean_combinatorial_list.count(True) == len(self)

    def _is_valid_indices(self, indices):
        return indices

    def __mul__(self, other_indices):
        
        A = self.index_objects
        B = other_indices.index_objects

        # Remove both sumed indices from result if indices are summed over.
        # Remove only last index if they are repeated.
        for (i, j) in list(it.product(self.index_objects, other_indices.index_objects)):
            if len(i*j) > 0:
                A.remove(i)
                B.remove(j)
            elif len(i+j) > 0:
                B.remove(j)
        Result = A + B
        
        # Re-enter orders / locations w.r.t new tensor indices
        for i in range(len(Result)):
            Result[i].order = i

        return Result
    
    def __add__(self, other_indices):
        return Indices(self, self.basis)
    
    def __sub__(self, other_indices):
        return Indices(self, self.basis)

    def return_rank_structure_as_tuple(self):
        return (int([x.component_type for x in self.index_objects].count('contravariant')), int([x.component_type for x in self.index_objects].count('covariant')))
    
    def return_new_index(self, other_indices, order, running_index, component_type, index_value = None):
        list_of_indices= [i.index_symbol for i in self.index_objects] + [i.index_symbol for i in other_indices.index_objects]
        new_index_symbol_not_in_others = list(set(index_symbols_list) - set(list_of_indices))[0]
        return Index(new_index_symbol_not_in_others, order, running_index, component_type, self.basis, index_value)
    
    def __len__(self):
        return len(self.index_objects)
    
    def get_zero_tensor(self):
        return MutableDenseNDimArray.zeros(*self.tensor_component_shape)
    
    
R0 = GrIndex("_{a}", 0, Base)
R1 = GrIndex("_{b}", 1, Base)
R2 = GrIndex("^{a}", 2, Base)
R3 = GrIndex("_{c}", 3, Base)

S = Indices([R0, R1], Base)
F = Indices([R2, R3], Base)
R0.order = 1
S*F

[GrIndex Object:( 
                     'index string representation' : _{b}, 
                     'index symbol' : b, 
                     'order' : 0, 
                     'dimention' : 4, 
                     'running index' : True, 
                     'index value(s)' : [0, 1, 2, 3], 
                     'component type' : covariant 
                 )
                 ,
 GrIndex Object:( 
                     'index string representation' : _{c}, 
                     'index symbol' : c, 
                     'order' : 1, 
                     'dimention' : 4, 
                     'running index' : True, 
                     'index value(s)' : [0, 1, 2, 3], 
                     'component type' : covariant 
                 )
                 ]

# Indices Class

In [25]:
class GrIndices(Indices):
    
    """
    Note before alteration of this class:
    
    Do not create a dependence between this class and any of the deserializing classes.
    
    The responsibility of the serializing classes is to decompose the information from strings,
    of different formats. Then to pass those information into this Indices class.
    
    
    
    Index = {
                'index' : '_{a}',
                'order' : 0,
                'symbol' : 'a',
                'running index' : True,
                'constant' : None,
                'index space' : 'cotangent',
                'component type' : 'covariant',
                'dimention' : 4
            }

    Index = {
                'index' : '_{b}',
                'order' : 1,
                'symbol' : 'b',
                'running index' : True,
                'constant' : None,
                'index space' : 'cotangent',
                'component type' : 'covariant',
                'dimention' : 4
            }

    indices = {
                'indices string representation' : '_{a}_{b}',
                'index objects' : [Index, Index],
                'indices slice' : slice,
                'dimention' : 4,
                'rank' : (0,2),
                'scalar' : False,
                'tensor is covariant' : bool,
                'summed indices' : []
            }
    """
    def __init__(self, indices, basis):
        
        # ----- Public ------
        self.indices_string_representation = self._is_valid_indices(indices)
        if isinstance(basis, str):
            self.basis = SympyParser(basis).convertToSympyObject()
        else:
            self.basis = basis
        Indices.__init__(self, self.return_index_objects(), self.basis)
        
    def return_index_objects(self):
        lis = []
        individual_indices = [item for item in re.split('(?=[_^])', self.indices_string_representation) if item]
        for i in range(len(individual_indices)):
            lis.append(GrIndex(individual_indices[i], i, self.basis))
        return lis
            
    def __repr__(self):
        """
        Explain method.
        """
        return f"""GrIndex Object:( 
                    'index string representation' : {self.indices_string_representation}, \n\
                    'index objects' : {self.index_objects}, \n\
                    'Coordinate basis' : {self.basis}, \n\
                    'is valid index structure' : {self.is_valid_index_structure}, \n\
                    'dimention' : {self.dimention}, \n\
                    'rank' : {self.rank_as_tuple}, \n\
                    'tensor components shape' : {self.tensor_component_shape}, \n\
                    'is a scalar' : {self.is_scalar} \n\
                )
                """

    def return_slices(self):
        return tuple([x.slc() for x in self.return_index_instances()])

    def _is_valid_indices(self, indices):
        return indices    

    def get_tensor_index(self, index_key):
        address = self._index_address.get(index_key)
        if not address:
            raise ValueError(index_key)
        return address

    def metric_product_from_index_structure(self):
        index_list = [item for item in re.split('(?=[_^])', self.indices_string_representation) if item]
        List_ = self.indices_string_representation
        list_metric_indices = []
        list_originaltensor = []
        #for i in self.return_index_instances():
        #    if i.is_contravariant:
        return list_metric_indices
    
    def return_new_index(self, indices, List):
        li = [i for i in indices]
        contra = "^{" + [i for i in List if i not in li][0] + "}"
        cov = "_{" + [i for i in List if i not in li][0] + "}"
        return [contra, cov]
    
# "_{a}_{b}_{c}_{d}" -> no changes
# "^{a}_{b}_{c}_{d}" -> ["^{a}^{f}"] ["_{f}_{b}_{c}_{d}"]
# "^{a}^{b}_{c}_{d}" -> 
A = GrIndices("^{a}^{b}_{theta = 0}_{phi=1}", '[t, r, theta, phi]')
B = GrIndices("_{theta}^{b}^{b}", '[t, r, theta, phi]')
C = A.index_objects + B.index_objects

#[(IndexA, IndexB) for (IndexA, IndexB) in list(it.product(A,B)) if itemgetter(*[0])(IndexA) == itemgetter(*[1])(IndexB)]
#[i for i in A]
#[RimannCov[i] for i in A]
smp.shape(RimannCov)
As = MutableDenseNDimArray.zeros(*smp.shape(RimannCov))
for i in A:
    As[i] = RimannCov[i]
    
A.__dict__

{'indices_string_representation': '^{a}^{b}_{theta = 0}_{phi=1}',
 'basis': [t, r, theta, phi],
 'index_objects': [GrIndex Object:( 
                      'index string representation' : ^{a}, 
                      'index symbol' : a, 
                      'order' : 0, 
                      'dimention' : 4, 
                      'running index' : True, 
                      'index value(s)' : [0, 1, 2, 3], 
                      'component type' : contravariant 
                  )
                  ,
  GrIndex Object:( 
                      'index string representation' : ^{b}, 
                      'index symbol' : b, 
                      'order' : 1, 
                      'dimention' : 4, 
                      'running index' : True, 
                      'index value(s)' : [0, 1, 2, 3], 
                      'component type' : contravariant 
                  )
                  ,
  GrIndex Object:( 
                      'index string representation' : _{theta=0}, 
  

In [26]:
class IndexProduct:
    """
    Make this class dependent on Indices Base class which has the data strcuture of all the base components we need in order to calculate the thing we want.
    The goal here is to make all classes completly independeny of the "_{a}^{b}" implementation. This string encodes the inforamtion we need in order to fill in
    the base class data. The base class AND the class which makes the conversion both can be put in as arguments to this class.
    """
    def __init__(self, indices1, indices2, operation):
        self.indices1 = indices1
        self.indices2 = indices2
        self.operation = self._operation_is_valid(operation)
        self.resultant_indices = self.dic()[self.operation]
        self.is_valid_tensor_expression = self._is_valid_tensor_expression()[self.operation]
        self.transpose_resulting_indices = self.transpose_list(self.resultant_indices)
        if self.is_valid_tensor_expression:
            self.indices_result = Indices(self.indices1*self.indices2, indices1.basis)
        else:
            raise ValueError("The indices and operation you have entered do not amount to a valid Tesnor expression. Please enter a valid tensor expression.")
    
    def _is_valid_tensor_expression(self):
        return {'*' : self.dic()['+'] == [],
                '+' : self.indices1 == self.indices2,
                '-' : self.indices1 == self.indices2}
    
    def dic(self):
        return {'*' : [i*j for i in self.indices1.index_objects for j in self.indices2.index_objects if len(i*j) > 0],
                '+' : [i+j for i in self.indices1.index_objects for j in self.indices2.index_objects if len(i+j) > 0],
                '-' : [i+j for i in self.indices1.index_objects for j in self.indices2.index_objects if len(i+j) > 0]}
    
    def _operation_is_valid(self, op):
        if op.replace(' ', '') in ['*', '-', '+']:
            return op.replace(' ', '')
        else:
            raise ValueError('Opperation input must be one of the following strings: + or - or *')
    
    def expression_is_covariant(self):
        boolean = True
        if self.operation == "*" and len(self.return_list_of_repeated_indices()) > 0:
            boolean = False
        elif self.operation == "-" and len(self.return_list_of_indices_to_sum_over()) > 0:
            boolean = False
        elif self.operation == "+" and len(self.return_list_of_indices_to_sum_over()) > 0:
            boolean = False
        return boolean
    
    def get_list(self):
        ListA = []
        ListB = []
        for i in self.indices1.index_objects:
            for j in self.indices_result.index_objects:
                if i == j:
                    ListA.append(i.order)
        for i in self.indices2.index_objects:
            for j in self.indices_result.index_objects:
                if i == j:
                    ListB.append(i.order)
        return [ListA, ListB]
    
    def transpose_list(self, l):
        """
        This will transpose a list.
        Note: This might be a function used else where, so might be good to place this in a helper file.
        
        Input : [[1,2],[3,4],[5,6]]
        Output: [[1,3,5],[2,4,6]]
        """
        return list(map(list, zip(*l)))
    
    """
    The following needs to work for:
    
    '+' : Check whether it is a correct expression and return the correct order, specified by the tensors indices.
    '-' : Same story as above.
    '*' : 
    """
    
    def ans_indices_locations(self):
        if self.operation == '*':
            return ([i for i in range(len(self.indices1)) if i not in self.transpose_resulting_indices[0]], [i for i in range(len(self.indices2)) if i not in self.transpose_resulting_indices[1]])
        else:
            raise ValueError('This function does not apply to any operation other than: *')
    
    def return_combinatorial_indices(self):
        #[(IndexA, IndexB) for (IndexA, IndexB) in list(it.product(self.indices1,self.indices2)) if itemgetter(*self.transpose_resulting_indices)(IndexA) == itemgetter(*self.transpose_resulting_indices)(IndexB)]
        return [(IndexA, IndexB) for (IndexA, IndexB) in list(it.product(self.indices1,self.indices2)) if itemgetter(*self.transpose_resulting_indices[0])(IndexA) == itemgetter(*self.transpose_resulting_indices[1])(IndexB)]
    
    def return_indices_array_for_component(self, components = None):
        """
        The following needs to work for:
        '+' : Check whether it is a correct expression and return the correct order, specified by the tensors indices.
        '-' : Same story as above.
        '*' : 
        Clumsy needs to be redone more elegantly. But for now it works and will return:
        Tensor Product
        Scalars
        Sums and subtractions
        All based on the indices you have done.
        """
        if not self.indices_result.is_scalar and components != None and self.operation == '*':
            _ = self.get_list()
            _1 = [i for i in range(len(_[0]))]
            _2 = [i for i in range(len(_[0]), len(_[0]) + len(_[1]))]
            return [(IndicesA, IndicesB) for (IndicesA, IndicesB) in self.return_combinatorial_indices() if itemgetter(*_[0])(IndicesA) == itemgetter(*_1)(components) \
                    and itemgetter(*_[1])(IndicesB) == itemgetter(*_2)(components)]
        
        elif not self.indices_result.is_scalar and components != None and self.operation in ['+', '-']:
            return [(IndicesA, IndicesB) for (IndicesA, IndicesB) in self.return_combinatorial_indices() if IndicesA == tuple(components)]

        else:
            return self.return_combinatorial_indices()

A = GrIndices("_{a}_{b}", '[t, r, theta, phi]')
B = GrIndices("^{a}^{b}", '[t, r, theta, phi]')
l = IndexProduct(A, B, "*")
# short circuits at shortest nested list if table is jagged:


In [27]:
class GrTensorProduct(IndexProduct):
    
    def __init__(self, 
                 TensorA, 
                 TensorB,
                 operation : str
                ):
        self.TensorA = TensorA
        self.TensorB = TensorB
        self.operation = operation
        IndexProduct.__init__(self, 
                                 self.TensorA.tensor_indices, 
                                 self.TensorB.tensor_indices,
                                 self.operation
                                )
        
    def compute_component(self):
        return {'*' : self.return_mult_component,
                '+' : self.return_add_component,
                '-' : self.return_sub_component}
        
    def return_mult_component(self, AnswerIndex):
        A = self.TensorA.all_components
        B = self.TensorB.all_components
        list_of_relevant_indices = self.return_indices_array_for_component(AnswerIndex)
        return sum([A[Indices[0]]*B[Indices[1]] for Indices in list_of_relevant_indices])
    
    def return_add_component(self, AnswerIndex):
        A = self.TensorA.all_components
        B = self.TensorB.all_components
        list_of_relevant_indices = self.return_indices_array_for_component(AnswerIndex)
        return sum([A[Indices[0]]+B[Indices[1]] for Indices in list_of_relevant_indices])
    
    def return_sub_component(self, AnswerIndex):
        A = self.TensorA.all_components
        B = self.TensorB.all_components
        list_of_relevant_indices = self.return_indices_array_for_component(AnswerIndex)
        return sum([A[Indices[0]]-B[Indices[1]] for Indices in list_of_relevant_indices])

    def return_tensor(self):
        zero_tensor = self.indices_result.get_zero_tensor()
        computed_component = self.compute_component()[self.operation]
        for i in self.indices_result:
            zero_tensor[i] = computed_component(i)
        return GrTensor(zero_tensor, self.indices_result)


#GrTensorProduct(X, Y, '*').return_tensor()

In [28]:
class GrTensor:
    def __init__(self, components, indices, name : str = 'none', basis = None):
        
        """
        Parameters:
            components : type = string OR sympy MutableDenseNDimArray
            basis : type = string OR sympy MutableDenseNDimArray
            indices : type = GrIndices
        
        ---------------
        
        Optional: 
            name: type = string
        """
        self.tensor_indices = IndexCategorizer(indices, basis if isinstance(indices,str) else indices.basis ).indices_object
        self.basis = self.tensor_indices.basis
        self.all_components = components
        self.components = self.all_components[self.tensor_indices._]
        self.name = name
        if self.tensor_indices.is_scalar:
            self.shape = 1
            self.rank = self.shape
        else:
            self.shape = smp.shape(self.components)
            self.rank = len(self.shape)
        self.dimention = self.tensor_indices.dimention
        
    def __add__(self, other):
        return GrTensorProduct(self, other, "+").return_tensor()
    
    def __radd__(self, other):
        return GrTensorProduct(other, self, "+").return_tensor()
    
    def __mul__(self, other):
        if isinstance(other, float) or isinstance(other, int):
            return GrTensor(other*self.components, self.basis, self.tensor_indices)
        elif isinstance(other, GrTensor):
            return GrTensorProduct(self, other, "*").return_tensor()
        else:
            raise ValueError("Object {} is not compatible with operation * with a GrTensor object.".format(other))
        
    def __rmul__(self, other):
        if isinstance(other, float) or isinstance(other, int):
            return GrTensor(other*self.components, self.tensor_indices)
        elif isinstance(other, GrTensor):
            return GrTensorProduct(other, self, "*").return_tensor()
        else:
            raise ValueError("Object {} is not compatible with operation * with a GrTensor object.".format(other))
    
    def __sub__(self, other):
        return GrTensorProduct(self, other, "-").return_tensor()
    
    def __rsub__(self, other):
        return GrTensorProduct(other, self, "-").return_tensor()
    
    def __truediv__(self, other):
        if type(other) == float or type(other) == int:
            return GrTensor(self.components/other,self.tensor_indices)
        else:
            raise ValueError("Cannot divide with anything other than int or float.")
            
    def gr_tensor_as_dict(self):
        return {
                'name' : self.name,
                'components' : str(self.components),
                'basis' : str(self.basis),
                'dimention' : self.dimention,
                'indices' : GrIndices(self.indices).indices_as_dict()
                }
    
    def return_tensor_ans_with_numbers(self, number):
        D = self.dimention
        A = int(self.rank)
        Shape = self.return_tensor_ans_shape()
        return smp.MutableDenseNDimArray(smp.ones(D**A)*number, Shape)
    

#f = MetricTensor(Metric, '_{a}_{b}', Base)*MetricTensor(InvMetric, '^{a}_{c}', Base)
#[item for item in re.split('(?=[_^])', '_{a}_{b}') if item]
#[1 for i in range(4)]
#smp.diag(*[1,1,1,1])
#f = GrTensor(Metric,'[t, r, theta, phi]', '_{a}_{b}')*GrTensor(InvMetric,'[t, r, theta, phi]', '^{b}^{c}')
#f.components
# R^{a=0}_{b}_{c}_{d}
"""
Goal:

    GrTensor(Metric, GrIndices('_{a}_{b}', '[t, r, theta, phi]')).change_coordinates(Transformation('[t = t1, x = r, y = theta, z = phi]'))
"""
Ta = GrTensor(Metric, "_{a}_{b}", basis = '[t, r, theta, phi]')
Tb = GrTensor(Metric, "_{c}^{a}",basis = '[t, r, theta, phi]')
Tc = GrTensor(InvMetric, "^{a}^{b}",basis = '[t, r, theta, phi]')
c = GrTensor(Metric, "_{a}_{b}", basis = '[t, r, theta, phi]') + GrTensor(Metric, "_{b}_{a}",basis = '[t, r, theta, phi]')
F = Ta*Tb
GrTensor(Metric, "_{a}_{b}", basis = '[t, r, theta, phi]')*GrTensor(Metric, "_{c}^{a}",basis = '[t, r, theta, phi]')

In [29]:
from collections import namedtuple
class StringToMathJson:
    def __init__(self, string):
        self.string = string
        self.OpBehaviour = namedtuple('OpBehaviour', 'priority lmbd')
        self.operations = {"+": self.OpBehaviour(0, lambda x, y: y+x),
                           "-": self.OpBehaviour(0, lambda x, y: y-x),
                           "/": self.OpBehaviour(1, lambda x, y: y/x),
                           "*": self.OpBehaviour(1, lambda x, y: y*x),
                           "^": self.OpBehaviour(2, lambda x, y: y**x)}

    def tokenize(self):
        string = self.string.replace(' ', '')
        return [ i for i in re.split('(\+|\-|\(|\)|\*|\/)', string) if i]

    def match_tensors(self, i):
        string = i
        pattern = lambda x : "([a-zA-Z]+)([_^]\{[a-zA-Z]+\}|[_^]\{[a-zA-Z]+\=[0-9]}){" + str(x) + "}(?=(\*|\)|\+|\-|\/|$))"
        Total = [[x for x in re.finditer(pattern(j), string)] for j in range(1, 11)]
        return [tensor.group() for nested in Total for tensor in nested]

    def match_operators(self, i):
        """
            Make checks as to what the name of the function is:
                - Integrate
                - solve
                - diff
                - subs
                or just declared functions
                - f(x)
                - etc...
            Then, make checks on the names and structure of the parameters.
            And finally parse the object into the relevant sympy object and return it into the MathJSON object.
        """
        Input = i.replace(' ', '')
        Function = '(?<![a-zA-Z])' + '([a-zA-Z]+)' + '(\(([a-z]+\))' + '|' + '\(([a-z]+\,)*[a-z]\))'
        return [x for x in re.finditer(Function, Input)]

    def json_wrapped_token(self):
        tokens = self.tokenize()
        for i in range(len(tokens)):
            if tokens[i] not in ['+','-','/','(',')','*']:
                if bool(self.match_operators(tokens[i])):
                    tokens[i] = MathJSON({"operators" : tokens[i]})
                elif bool(self.match_tensors(tokens[i])):
                    tokens[i] = MathJSON({"tensor_string_representation" : tokens[i]})
                elif bool(re.match('[0-9]+', tokens[i].replace('.','',1))):
                    tokens[i] = MathJSON({"number" : tokens[i]})
        return tokens

    def to_rpn(self):
        tokens = self.json_wrapped_token()
        rpn_tokens = []
        op_stack = []

        for token in tokens:
            # Add number to rpn tokens
            if isinstance(token, MathJSON):
                rpn_tokens.append(token)
            # Add opening bracket to operation stack
            elif token == "(":
                op_stack.append(token)
            # Consumes all operations until matching opening bracket
            elif token == ")":
                while op_stack[-1] != "(":
                    rpn_tokens.append(op_stack.pop())
                op_stack.pop()
            elif token in list(self.operations.keys()):
                try:
                    # Check if we have operations that have higher priority on
                    # the op_stack and add them to rpn_tokens so that they are evaluated first:
                    token_priority = self.operations[token].priority
                    while op_stack[-1] != "(" and self.operations[op_stack[-1]].priority >= token_priority:
                        rpn_tokens.append(op_stack.pop())
                except IndexError:  # op_stack is empty
                    pass
                # Add the current operation to the op_stack:
                op_stack.append(token)

        # Add remaining operations to rpn tokens
        while len(op_stack) != 0:
            rpn_tokens.append(op_stack.pop())

        return rpn_tokens

    def calculate(self):
        rpn_tokens = self.to_rpn()
        val_stack = []

        for token in rpn_tokens:
            if isinstance(token, MathJSON):
                val_stack.append(token)
            elif token in list(self.operations.keys()):
                args = []
                for x in range(self.operations[token].lmbd.__code__.co_argcount):
                    # If this throws an error user didn't give enough args
                    args.append(val_stack.pop())
                result = self.operations[token].lmbd(*args)
                val_stack.append(result)

        # If the value stack is bigger than one we probably made an error with the input
        assert len(val_stack) == 1
        return val_stack[0]

In [30]:
class MathJSON:
    def __init__(self, objectJson):
        self.objectJson = objectJson
        
    def __add__(self, other):
        return MathJSON({ 'Add' : [self.objectJson, other.objectJson] })
    
    def __mul__(self, other):
        return MathJSON({ 'Mul' : [self.objectJson, other.objectJson] })
    
    def __sub__(self, other):
        return MathJSON({ 'Sub' : [self.objectJson, other.objectJson] })
    
    def __truediv__(self, other):
        return MathJSON({ 'Div' : [self.objectJson, other.objectJson] })


In [31]:
h = MathJSON(GrTensor(Metric, "_{a}_{b}", basis = '[t, r, theta, phi]'))*MathJSON(GrTensor(Metric, "_{c}^{a}",basis = '[t, r, theta, phi]'))


In [32]:
class TensorStringRepresentations:

    def __init__(self, tensor_string):
        self.database = {'G' : { 'Components' : Metric , 'Basis' : Base} }
        self.tensor_string = tensor_string
        self.tensor_object = GrTensor(self.database[self.representation_one()['name']]['Components'], GrIndices(self.representation_one()['indices'], self.database[self.representation_one()['name']]['Basis']))

    def representation_one(self):
        string = self.tensor_string
        name = re.match('([a-zA-Z]+)', string).group()
        indices = string.replace(name, '')
        return {'name' : name, 'indices' : indices}

class ComputeMathJson:
    def __init__(self, x):
        database = {}
        self.operations = {"Add" : lambda List : ComputeMathJson(List[0])+ComputeMathJson(List[1]),
                           "Mul" : lambda List : ComputeMathJson(List[0])*ComputeMathJson(List[1]),
                           "Sub" : lambda List : ComputeMathJson(List[0])-ComputeMathJson(List[1]),
                           "Div" : lambda List : ComputeMathJson(List[0])/ComputeMathJson(List[1]),
                           "Diff" : lambda List : smp.diff(List[0], List[1]),
                           "Int" : lambda List : smp.integrate(List[0], List[1]),
                           "Eq" : lambda List : database.update({List[0] : ComputeMathJson(List[1]).x}),
                           "integer" : lambda x : int(x),
                           "Pow" : lambda List : ComputeMathJson(List[0])**ComputeMathJson(List[1]),
                           "tensor_string_representation" : lambda x : TensorStringRepresentations(x).tensor_object,
                           "sympy_array" : lambda x : MutableDenseNDimArray(self.Generate(x)),
                           "sympy_variable" : lambda x : smp.symbols(x),
                           "sympy_function" : lambda List : smp.symbols(List[0] , cls = smp.Function)(*tuple([ComputeMathJson(List[1][i]).x for i in range(len(List[1]))]))}
        
        if isinstance(x, dict):
            if [i for i in x][0] == "Diff":
                self.x = self.operations["Diff"](x["Diff"])
            elif [i for i in x][0] == "sympy_variable":
                self.x = self.operations["sympy_variable"](x["sympy_variable"])
            elif [i for i in x][0] == "sympy_function":
                self.x = self.operations["sympy_function"](x["sympy_function"])
            elif [i for i in x][0] == "sympy_array":
                self.x = self.operations["sympy_array"](x["sympy_array"])
            elif [i for i in x][0] == "Int":
                self.x = self.operations["Int"](x["Int"])
            elif [i for i in x][0] == "Pow":
                self.x = self.operations["Pow"](x["Pow"])
            elif [i for i in x][0] == "integer":
                self.x = self.operations["integer"](x["integer"])
            elif [i for i in x][0] == "tensor_string_representation":
                self.x = self.operations["tensor_string_representation"](x["tensor_string_representation"])
            else:
                self.x = self.operations[[i for i in x][0]](x[[i for i in x][0]])
        else:
            self.x = x
            
    def Generate(self, arr):
        IterableComponentsForArray = lambda Array : list(it.product(np.arange(np.array(Array).shape[0]), repeat = len(np.array(Array).shape)))
        new = np.array(arr)
        for i in IterableComponentsForArray(arr):
            new[i] = ComputeMathJson(np.array(arr)[i]).x
        return new
            
    def __add__(self, other):
        return self.x + other.x
    
    def __mul__(self, other):
        return self.x*other.x
    
    def __sub__(self, other):
        return self.x-other.x
    
    def __truediv__(self, other):
        return self.x/other.x

    def __pow__(self, other):
        return self.x**other.x

In [33]:

# Challanges:
# "F(x)" ->  MathJSON({'sympy_function' : ['F' , ({'sympy_variable' : 'x'})]})

# MathJSON({'sympy_function' : ['f' , ({'sympy_variable' : 'x'}, {'sympy_variable' : 'y'})]}).objectJson

array = MathJSON({'sympy_array' : [x, y]}).objectJson

MutableDenseNDimArray([[1,1,1],[2,2,2],[1,1,1]]).shape[0]

3

In [34]:
IterableComponentsForArray = lambda Array : list(it.product(np.arange(Array.shape[0]), repeat = len(Array.shape)))


In [35]:
array = [[{'sympy_function' : ['f' , [{'sympy_variable' : 'x'}] ]}, {'sympy_function' : ['f' , [{'sympy_variable' : 'x'}] ]}],[{'sympy_function' : ['f' , [{'sympy_variable' : 'x'}] ]}, {'sympy_function' : ['f' , [{'sympy_variable' : 'x'}] ]}]]
ar = MutableDenseNDimArray([[1,1],[2,2]])

[ar[i] for i in IterableComponentsForArray(ar)]

def Generate(arr):
    IterableComponentsForArray = lambda Array : list(it.product(np.arange(np.array(Array).shape[0]), repeat = len(np.array(Array).shape)))
    new = np.array(arr)
    for i in IterableComponentsForArray(arr):
        new[i] = ComputeMathJson(np.array(arr)[i]).x
    return new
MutableDenseNDimArray(Generate(array))


[[f(x), f(x)], [f(x), f(x)]]

In [36]:
class StringArrayToMathJSON:
    def __init__(self, string_array):
        if isinstance(string_array, str):
            self.string_array = string_array
        else:
            raise ValueError("Expected string as argument.")

    def to_mathJson(self):
        NumpyArray = self.numpyfy_string_array()
        IterableComponentsForArray = lambda Array : list(it.product(np.arange(np.array(Array).shape[0]), repeat = len(np.array(Array).shape)))
        return np.array([StringToMathJson(NumpyArray[i]).calculate().objectJson for i in IterableComponentsForArray(NumpyArray)]).reshape(NumpyArray.shape).tolist()

    def numpyfy_string_array(self):
        StringArray = self.string_array
        empty_string_array = re.sub('[^\[\]\,]','1', StringArray)
        return np.array(StringArray.replace('[','').replace(']','').split(',')).reshape(*np.array(eval(empty_string_array)).shape)

StringArrayToMathJSON('[[G^{a}^{b}*G_{a}_{b},G^{a}^{b}],[G^{a}^{b},1]]').to_mathJson()

[[{'Mul': [{'tensor_string_representation': 'G^{a}^{b}'},
    {'tensor_string_representation': 'G_{a}_{b}'}]},
  {'tensor_string_representation': 'G^{a}^{b}'}],
 [{'tensor_string_representation': 'G^{a}^{b}'}, {'number': '1'}]]

In [37]:
re.sub('(?<=\,|\[)(\,)(?=\,|\])','', re.sub('[^\[\]\,]', '', '[1,1]'))

'[]'

Metric = [[r, k],[wdwdn, wdwd]]
Inverse = [[sf],[sfsf]]

Metric_{a}_{b}*Inverse^{a}^{b}
 
Workflow()


In [38]:
ComputeMathJson({'sympy_array' : [[{'sympy_variable' : 'x' }, {'sympy_function' : ['f' , [{'sympy_variable' : 'x'}] ]}],[{'sympy_function' : ['f' , [{'sympy_variable' : 'x'}] ]}, {'sympy_function' : ['f' , [{'sympy_variable' : 'x'}] ]}]]
 }).x

[[x, f(x)], [f(x), f(x)]]

In [39]:
input_box = 'G^{a}^{b}*G_{a}_{b}'
ComputeMathJson(StringToMathJson(input_box).calculate().objectJson).x.components
StringToMathJson(input_box).calculate().objectJson

{'Mul': [{'tensor_string_representation': 'G^{a}^{b}'},
  {'tensor_string_representation': 'G_{a}_{b}'}]}

In [40]:
class GrTensorOld(Tensor, GrIndices):
    def __init__(self, components, basis, indices, name : str = 'none'):
        Tensor.__init__(self, components, basis)
        if isinstance(indices, str):
            GrIndices.__init__(self, indices)
            self.tensor_indices = GrIndices(indices)
        elif isinstance(indices, GrIndices):
            self.tensor_indices = indices
        self.name = name
        self.components = components
        self.shape = smp.shape(self.components)
        self.dimention = int(self.shape[0])
        self.rank = len(self.shape)
        self.index_instace_list = self.return_index_instances()
        
    def __add__(self, other):
        expression = GrTensorArithmetic(GrTensor(self.components, self.basis,self.indices),GrTensor(other.components,other.basis, other.indices))
        return GrTensor(expression.return_tensor("+"),self.basis,expression.tensor_ans_index_as_string)
    
    def __radd__(self, other):
        expression = GrTensorArithmetic(GrTensor(self.components, self.basis,self.indices),GrTensor(other.components,other.basis, other.indices))
        return GrTensor(expression.return_tensor("+"),self.basis,expression.tensor_ans_index_as_string)
    
    def __mul__(self, other):
        if isinstance(other, float) or isinstance(other, int):
            return GrTensor(other*self.components,self.basis,self.indices)
        elif isinstance(other, GrTensor):
            expression = GrTensorProduct(GrTensor(self.components, self.basis,self.indices),GrTensor(other.components, other.basis,other.indices))
            return GrTensor(expression.return_tensor(),self.basis,expression.tensor_ans_index_as_string)
        else:
            raise ValueError("Object {} is not compatible with operation * with a GrTensor object.".format(other))
        
    def __rmul__(self, other):
        if isinstance(other, float) or isinstance(other, int):
            return GrTensor(other*self.components,self.basis,self.indices)
        elif isinstance(other, GrTensor):
            expression = GrTensorProduct(GrTensor(self.components, self.basis, self.indices),GrTensor(other.components, other.basis, other.indices))
            return GrTensor(expression.return_tensor(),self.basis,expression.tensor_ans_index_as_string)
        else:
            raise ValueError("Object {} is not compatible with operation * with a GrTensor object.".format(other))
    
    def __sub__(self, other):
        expression = GrTensorArithmetic(GrTensor(self.components, self.basis,self.indices),GrTensor(other.components,other.basis, other.indices))
        return GrTensor(expression.return_tensor("-"),self.basis,expression.tensor_ans_index_as_string)
    
    def __rsub__(self, other):
        return self.__sub__()
    
    def __truediv__(self, other):
        if type(other) == float or type(other) == int:
            return GrTensor(self.components/other,self.indices)
        else:
            raise ValueError("Cannot divide with anything other than int or float.")
            
    def gr_tensor_as_dict(self):
        return {
                'name' : self.name,
                'components' : str(self.components),
                'basis' : str(self.basis),
                'dimention' : self.dimention,
                'indices' : GrIndices(self.indices).indices_as_dict()
                }
    
    def return_tensor_ans_with_numbers(self, number):
        D = self.dimention
        A = int(self.rank)
        Shape = self.return_tensor_ans_shape()
        return smp.MutableDenseNDimArray(smp.ones(D**A)*number, Shape)
#f = MetricTensor(Metric, '_{a}_{b}', Base)*MetricTensor(InvMetric, '^{a}_{c}', Base)
#[item for item in re.split('(?=[_^])', '_{a}_{b}') if item]
#[1 for i in range(4)]
#smp.diag(*[1,1,1,1])
f = GrTensor(Metric,'[t, r, theta, phi]', '_{a}_{b}')*GrTensor(InvMetric,'[t, r, theta, phi]', '^{b}^{c}')
#f.components
f = GrTensor(Metric,'[t, r, theta, phi]', "_{a}_{b}")

AttributeError: 'IndexCategorizer' object has no attribute 'indices_object'

In [ ]:
class MetricTensor(GrTensor, GrIndices):
    def __init__(self, components, basis, indices):
        GrIndices.__init__(self, indices)
        GrTensor.__init__(
            self, 
            components, 
            basis,
            indices, 
            'metric')
        self.basis = basis
        self.input_components = components
        self.inverse_metric_components = self.return_inverse_metric(self.components)
        self.metric_components = self.return_metric(self.components)
    
    def invert_components(self, components):
        N = len(self.basis)
        g_m = components.tomatrix()
        inv_g = g_m.inv()
        A = smp.MutableDenseNDimArray(smp.zeros(N**2),(N,N))
        for i in range(N):
            for j in range(N):
                A[i,j] = inv_g[i, j]
        return smp.simplify(A)
    
    def return_inverse_metric(self, components):
        if str(self.return_rank_structure_as_tuple()) == '(2, 0)':
            return self.invert_components(components)
        elif str(self.return_rank_structure_as_tuple()) == '(0, 2)':
            return self.components
            
    def return_metric(self, components):
        if str(self.return_rank_structure_as_tuple()) == '(2, 0)':
            return components
        elif str(self.return_rank_structure_as_tuple()) == '(0, 2)':
            return self.invert_components(components)
    
    def _is_valid_indicesm(self, indices, basis, components):
        diag_form = smp.MutableDenseNDimArray(smp.diag(*[1 for i in range(len(basis))]))
        list_of_indices = [item for item in re.split('(?=[_^])', indices) if item]
        if int(len(list_of_indices)) == 2:
            if list_of_indices[0][0] != list_of_indices[1][0] and components != diag_form:
                raise ValueError("Invalid metric, the index you have entered must be unitary.")
            else:
                return indices
        else:
            raise ValueError("Your Metric tensor must have two indices.")
            
    def _is_valid_components(self, components, basis):
        if smp.shape(components) == (len(basis), len(basis)):
            return components
        else:
            raise ValueError("Invalid metric component form, please make sure the components you enter are NxN.")
            
Hello = MetricTensor(Metric, Base, "_{a}_{b}")
Hello.__dict__()

TypeError: __init__() missing 1 required positional argument: 'basis'

In [ ]:
class RimannCurvatureTensor(GrTensor, BaseFunctions, GrIndices):
    def __init__(self, Metric: MetricTensor, indices):
        GrIndices.__init__(self, indices)
        self.metric = self.__is_valid_Metric(Metric)
        self.MetricComponents = self.metric.metric_components
        self.Basis = self.metric.basis
        BaseFunctions.__init__(self, self.MetricComponents, self.Basis)
        self.CovarientRiemman = {
            '0000' : GrTensor(self.Riemann0000(),"_{a}_{b}_{c}_{d}")
        }
    def __is_valid_Metric(self, Metric):
        if type(Metric) == MetricTensor:
            return Metric
        else:
            raise ValueError("The Metric Parameter is not a MetricTensor object.")
        
        
hell = RimannCurvatureTensor(Hello, "_{a}_{b}_{c}_{d}")
#BaseFunctions(Hello.metric_components, Hello.basis)
hell.CovarientRiemman

TypeError: __init__() missing 1 required positional argument: 'indices'

# Derivative Class

In [ ]:
class DerivativeCalculation(BaseFunctions):
    def __init__(self, Metric, Basis, TensorObject, covariant = False):
        BaseFunctions.__init__(self, Metric, Basis)
        self.TensorObject = TensorObject
        self.covariant = covariant
    
    def return_derivative(self,rank_tuple):
        derivative = { 
                'normal' : self.TDerivative,
                '(1, 1)' : self.CovariantD11,
                '(0, 1)' : self.CovariantD01,
                '(1, 0)' : self.CovariantD10,
                '(0, 2)' : self.CovariantD02,
                '(2, 0)' : self.CovariantD20
                }
        if self.covariant:
            return derivative[rank_tuple](self.TensorObject)
        else:
            return derivative['normal'](self.TensorObject)


class GrDerivative(GrIndices, BaseFunctions):
    def __init__(self, basis, index, metric = None, covariant = False):
        GrIndices.__init__(self, index)
        self.basis = basis
        self.dim = len(basis)
        self.index = index
        self.metric = metric
        self.covariant = covariant
        
    def __mul__(self, TensorObject):
        derivative  = DerivativeCalculation(self.metric, self.basis, TensorObject.components, self.covariant)
        return GrTensor(derivative.return_derivative(self.return_type_of_derivative(TensorObject)), self.sum_indices(GrIndices(TensorObject.indices)))
    
    def return_type_of_derivative(self, TensorObject):
        return str(TensorObject.rank_as_tuple)

    def answerIndex(self, other):
        add = other + self.index
        for i in other:
            for j in self.index:
                if i[0] == j[0] and i[1] != j[1]:
                    add.remove(i)
                    add.remove(j)
                elif i[0] == j[0] and i[1] == j[1]:
                    add.remove(i)
        return add

In [ ]:
class BaseTensorExpression(GrIndices):
    def __init__(self, A, B):
        self.A = A
        self.B = B
        self.tensor_a_components = self.A.components
        self.tensor_b_components = self.B.components
        self.tensor_a_shape = self.A.shape
        self.tensor_b_shape = self.B.shape
        self.tensor_a_dimention = self.A.dimention
        self.tensor_a_dimention = self.B.dimention
        self.tensor_a_rank = self.A.rank
        self.tensor_b_rank = self.B.rank
        self.tensor_a_list_of_index_instances = A.index_instace_list
        self.tensor_b_list_of_index_instances = B.index_instace_list
        self.tensor_ans_index_as_string = self.return_resulting_indices_as_string()
        self.tensor_ans_list_of_index_instances = self.return_index_instances()
        self.tensor_ans_rank = len(self.tensor_ans_list_of_index_instances)
        self.tensor_ans_zero_components = self.return_tensor_ans_with_zeros()
        self.tensor_ans_shape = self.return_tensor_ans_shape()
        self.tensor_ans_list_of_index_combinatorics = self.return_tensor_ans_index_combinatorics_as_list()

    def return_index_instances(self):
        instance_list = []
        individual_indices = [item for item in re.split('(?=[_^])', self.tensor_ans_index_as_string) if item]
        for i in range(len(individual_indices)):
            instance_list.append(GrIndex(individual_indices[i],int(i)))
        return instance_list
    
    # Takes two lists of indices and is responsible for working out what the resulting index structure is:
    def return_resulting_indices_as_string(self):
        A_instances = self.tensor_a_list_of_index_instances
        B_instances = self.tensor_b_list_of_index_instances
        Total = A_instances + B_instances
        ListA = list(dict.fromkeys([x.index for x in Total]))
        for i in A_instances:
            for j in B_instances:
                if i == j and i.index[0] != j.index[0]:
                    ListA.remove(i.index)
                    ListA.remove(j.index)
        return ''.join(ListA)
    
    def return_tensor_ans_shape(self):
        D = self.tensor_a_dimention
        N = self.tensor_ans_rank
        Ans_shape = ()
        for i in range(N):
            y = list(Ans_shape)
            y.append(D)
            Ans_shape = tuple(y)
        return Ans_shape
    
    def return_tensor_ans_with_zeros(self):
        D = self.tensor_a_dimention
        A = int(self.tensor_ans_rank)
        Shape = self.return_tensor_ans_shape()
        return smp.MutableDenseNDimArray(smp.zeros(D**A), Shape)
    
    def return_tensor_ans_index_combinatorics_as_list(self):
        D = self.tensor_a_dimention
        Shape = self.return_tensor_ans_shape()
        return list(it.product(np.arange(0, D, 1), repeat = len(Shape)))
    
A = GrTensor(Metric,'[t, r, theta, phi]', '_{a}_{b}')
B = GrTensor(InvMetric,'[t, r, theta, phi]', '^{b}^{c}')

BaseTensorExpression(A, B).tensor_ans_index_as_string

'_{a}^{c}'

In [ ]:
class BaseTensorExpression(GrTensor):
    def __init__(self, tensor1, tensor2):
        GrIndices.__init__(self, )
        self.A = A
        self.B = B
        self.tensor1_indices = tensor1.tensor_indices*tensor2.tensor_indices
        self.tensor_a_components = self.A.components
        self.tensor_b_components = self.B.components
        self.tensor_a_shape = self.A.shape
        self.tensor_b_shape = self.B.shape
        self.tensor_a_dimention = self.A.dimention
        self.tensor_a_dimention = self.B.dimention
        self.tensor_a_rank = self.A.rank
        self.tensor_b_rank = self.B.rank
        self.tensor_a_list_of_index_instances = A.index_instace_list
        self.tensor_b_list_of_index_instances = B.index_instace_list
        self.tensor_ans_index_as_string = self.return_resulting_indices_as_string()
        self.tensor_ans_list_of_index_instances = self.return_index_instances()
        self.tensor_ans_rank = len(self.tensor_ans_list_of_index_instances)
        self.tensor_ans_zero_components = self.return_tensor_ans_with_zeros()
        self.tensor_ans_shape = self.return_tensor_ans_shape()
        self.tensor_ans_list_of_index_combinatorics = self.return_tensor_ans_index_combinatorics_as_list()

    
    def return_index_instances(self):
        instance_list = []
        indices_as_string = self.return_resulting_indices_as_string()
        individual_indices = [item for item in re.split('(?=[_^])', indices_as_string) if item]
        for i in range(len(individual_indices)):
            instance_list.append(GrIndex(individual_indices[i],int(i)))
        return instance_list
    
    # Takes two lists of indices and is responsible for working out what the resulting index structure is:
    def return_resulting_indices_as_string(self):
        ListA = list(dict.fromkeys([x.index for x in tensor1.index_instace_list + tensor2.index_instace_list]))
        for i in tensor1.index_instace_list:
            for j in tensor2.index_instace_list:
                if i == j and i.index[0] != j.index[0]:
                    ListA.remove(i.index)
                    ListA.remove(j.index)
        return ''.join(ListA)
    
    def return_tensor_ans_shape(self):
        D = self.tensor_a_dimention
        N = self.tensor_ans_rank
        Ans_shape = ()
        for i in range(N):
            y = list(Ans_shape)
            y.append(D)
            Ans_shape = tuple(y)
        return Ans_shape
    
    def return_tensor_ans_with_zeros(self):
        D = self.tensor_a_dimention
        A = int(self.tensor_ans_rank)
        Shape = self.return_tensor_ans_shape()
        return smp.MutableDenseNDimArray(smp.zeros(D**A), Shape)
    
    def return_tensor_ans_index_combinatorics_as_list(self):
        D = self.tensor_a_dimention
        Shape = self.return_tensor_ans_shape()
        return list(it.product(np.arange(0, D, 1), repeat = len(Shape)))

In [ ]:
# Keep an eye on: Multiple tensor products such as:
# G^{a}^{b}*G^{c}^{d}*R_{b}_{d}_{f}_{g} => R^{a}^{c}_{f}_{g}
# The tensor Multiplication G^{a}^{b}*G^{c}^{d} will currently not work

# ---- Solution one ----
# One simple but slower method would be to perform a tensor product and get the new tensor G^{a}^{b}^{c}^{d} out
# And then perfrom the tensor calculation G^{a}^{b}^{c}^{d}*R_{b}_{d}_{f}_{g}
# This would work fine but would be slower

#---- Solution Two ----
# Define some order within the multiplication symbols so that G^{c}^{d}*R_{b}_{d}_{f}_{g} is performed first, followed by 
# G^{a}^{b}*R_{b}^{c}_{f}_{g}

# ------ Target --------
# This class takes in a index produxt class and 
# [Tensor1[i[:Rank]]*Tensor2[i[Rank:]] for i in IndicesProduct ...] 

class GrTensorProduct(BaseTensorExpression):
    
    def __init__(self, A:GrTensor, B:GrTensor):
        BaseTensorExpression.__init__(self, A, B)
        self.list_of_index_locations_to_sum_over_wrt_each_tensor = self.return_list_of_indices_to_sum_over()
        self.list_of_all_tensor_index_concatinated_combinatorials = list(it.product(np.arange(0, self.tensor_a_dimention, 1), repeat = self.tensor_a_rank + self.tensor_b_rank))
        self.list_of_list_of_summed_index_locations_wrt_both_tensor_indices_concatinanted = [[i[0],i[1] + self.tensor_a_rank] for i in self.list_of_index_locations_to_sum_over_wrt_each_tensor]
        self.flat_list_of_summed_index_locations_wrt_both_tensor_indices_concatinanted = [item for sublist in self.list_of_list_of_summed_index_locations_wrt_both_tensor_indices_concatinanted for item in sublist]
        self.flat_list_of_ans_tensor_index_locations_wrt_both_tensor_indices_concatinanted = [i for i in list(np.arange(0, self.tensor_a_rank + self.tensor_b_rank, 1)) if i not in self.flat_list_of_summed_index_locations_wrt_both_tensor_indices_concatinanted]
        self.flat_list_of_tensor_a_summed_index_locations_wrt_both_tensor_indices_concatinanted = [i[0] for i in self.list_of_list_of_summed_index_locations_wrt_both_tensor_indices_concatinanted]
        self.flat_list_of_tensor_b_summed_index_locations_wrt_both_tensor_indices_concatinanted = [i[1] for i in self.list_of_list_of_summed_index_locations_wrt_both_tensor_indices_concatinanted]
        
    def return_list_of_indices_to_sum_over(self):
        Summed_indices = []
        A_instances = self.tensor_a_list_of_index_instances
        B_instances = self.tensor_b_list_of_index_instances
        for i in A_instances:
            for j in B_instances:
                if i == j and i.index[0] != j.index[0]:
                    Summed_indices.append([int(i.order), int(j.order)])
        return Summed_indices
    
    def return_component(self, AnswerIndex):
        A = self.tensor_a_components
        B = self.tensor_b_components
        IndexCombinatorials = self.list_of_all_tensor_index_concatinated_combinatorials
        index_locations_tensor_a = self.flat_list_of_tensor_a_summed_index_locations_wrt_both_tensor_indices_concatinanted
        index_locations_tensor_b = self.flat_list_of_tensor_b_summed_index_locations_wrt_both_tensor_indices_concatinanted
        ans_index_locations = self.flat_list_of_ans_tensor_index_locations_wrt_both_tensor_indices_concatinanted
        Rank = self.tensor_a_rank
        return sum([A[i[:Rank]]*B[i[Rank:]] for i in IndexCombinatorials if itemgetter(*index_locations_tensor_a)(i) == itemgetter(*index_locations_tensor_b)(i) and itemgetter(*ans_index_locations)(i) == AnswerIndex])
    
    def return_tensor(self):
        I = self.tensor_ans_list_of_index_combinatorics
        Tensor = self.tensor_ans_zero_components
        a = self.tensor_a_rank
        b = self.tensor_b_rank
        Shape = self.tensor_ans_shape
        for i in I:
            Tensor[i] = self.return_component(i)
        return Tensor
    
class GrTensorProductTest():
    
    def __init__(self, TensorA, TensorB):
        self.TensorA = TensorA
        self.TesnorB = TensorB
        self.IndexProduct = 
        
    def return_components(self, AnswerIndex):

In [ ]:
class GrTensorProductTest():
    
    def __init__(self, TensorA, TensorB):
        self.TensorA = TensorA
        self.TesnorB = TensorB
        self.IndexProduct = 
        
    def return_components(self, AnswerIndex):
        

In [ ]:
class GrTensorArithmetic(BaseTensorExpression):
    
    def __init__(self, A:GrTensor, B:GrTensor):
        BaseTensorExpression.__init__(self, A, B)
        self.list_of_repeated_index_locations_wrt_each_tensor = self.return_list_of_repeated_indices()
        self.list_of_all_tensor_index_concatinated_combinatorials = list(it.product(np.arange(0, self.tensor_a_dimention, 1), repeat = self.tensor_a_rank + self.tensor_b_rank))
        self.list_of_list_of_repeated_index_locations_wrt_both_tensor_indices_concatinanted = [[i[0],i[1] + self.tensor_a_rank] for i in self.list_of_repeated_index_locations_wrt_each_tensor]
        self.flat_list_of_repeated_index_locations_wrt_both_tensor_indices_concatinanted = [item for sublist in self.list_of_list_of_repeated_index_locations_wrt_both_tensor_indices_concatinanted for item in sublist]
        self.flat_list_of_ans_tensor_index_locations_wrt_both_tensor_indices_concatinanted = [i for i in list(np.arange(0, self.tensor_a_rank + self.tensor_b_rank, 1)) if i not in self.flat_list_of_repeated_index_locations_wrt_both_tensor_indices_concatinanted]
        self.flat_list_of_tensor_a_repeated_index_locations_wrt_both_tensor_indices_concatinanted = [i[0] for i in self.list_of_list_of_repeated_index_locations_wrt_both_tensor_indices_concatinanted]
        self.flat_list_of_tensor_b_repeated_index_locations_wrt_both_tensor_indices_concatinanted = [i[1] for i in self.list_of_list_of_repeated_index_locations_wrt_both_tensor_indices_concatinanted]
    
    def return_list_of_repeated_indices(self):
        Same_indices = []
        A_instances = self.tensor_a_list_of_index_instances
        B_instances = self.tensor_b_list_of_index_instances
        Total = A_instances + B_instances
        for i in A_instances:
            for j in B_instances:
                if i == j and i.index[0] == j.index[0]:
                    Same_indices.append([int(i.order), int(j.order)])
        return Same_indices
    
    def return_component(self, AnswerIndex, operation):
        A = self.tensor_a_components
        B = self.tensor_b_components
        IndexCombinatorials = self.list_of_all_tensor_index_concatinated_combinatorials
        index_locations_tensor_a = self.flat_list_of_tensor_a_repeated_index_locations_wrt_both_tensor_indices_concatinanted
        index_locations_tensor_b = self.flat_list_of_tensor_b_repeated_index_locations_wrt_both_tensor_indices_concatinanted
        Rank = self.tensor_a_rank
        if operation == "+":
            return [A[i[:Rank]]+B[i[Rank:]] for i in IndexCombinatorials if itemgetter(*index_locations_tensor_a)(i) == itemgetter(*index_locations_tensor_b)(i) and itemgetter(*index_locations_tensor_a)(i) == AnswerIndex]
        elif operation == "-":
            return [A[i[:Rank]]-B[i[Rank:]] for i in IndexCombinatorials if itemgetter(*index_locations_tensor_a)(i) == itemgetter(*index_locations_tensor_b)(i) and itemgetter(*index_locations_tensor_a)(i) == AnswerIndex]
    
    def return_tensor(self, operation):
        I = self.tensor_ans_list_of_index_combinatorics
        Tensor = self.tensor_ans_zero_components
        a = self.tensor_a_rank
        b = self.tensor_b_rank
        Shape = self.tensor_ans_shape
        for i in I:
            Tensor[i] = self.return_component(i, operation)[0]
        return Tensor


In [ ]:
iv = GrTensor(InvMetric, '^{theta}^{mu}')
riv = GrTensor(RimannCov, '_{theta}_{phi}_{mu}_{nu}')
new = GrTensor(RimannCov, '_{a}_{c}_{b}_{d}')*GrTensor(InvMetric, '^{a}^{u}')*GrTensor(InvMetric, '^{c}^{d}')

TypeError: __init__() missing 1 required positional argument: 'indices'

In [ ]:
GrTensorArithmetic(iv, iv).return_tensor("+")

NameError: name 'iv' is not defined

In [ ]:
from operator import itemgetter

# Works for when Answer tensor still has index i.e. not a scalar.
# Also only works when there are summed over indexes.
def Indexer(A, B, AnswerIndex, SummedIndices):
    A_shape = smp.shape(A)
    B_shape = smp.shape(B)
    a_dim = A_shape[0]
    b_dim = B_shape[0]
    a_index_len = int(len(A_shape))
    b_index_len = int(len(B_shape))
    NewSummedIndices = [[i[0],i[1] + a_index_len] for i in SummedIndices]
    FlatNewSummedIndices = [item for sublist in NewSummedIndices for item in sublist]
    IndicesLeftAsAnswer = [i for i in list(np.arange(0, a_index_len + b_index_len, 1)) if i not in FlatNewSummedIndices]
    ItemGetterListOne = [i[0] for i in NewSummedIndices]
    ItemGetterListTwo = [i[1] for i in NewSummedIndices]
    x_lis = list(it.product(np.arange(0, a_dim, 1), repeat = a_index_len + b_index_len))
    Answer = [[i[:a_index_len],i[a_index_len:]] for i in x_lis if itemgetter(*ItemGetterListOne)(i) == itemgetter(*ItemGetterListTwo)(i) and itemgetter(*ItemGetterListOne)(i) == AnswerIndex]
    return Answer

# '_{0}^{a}' => return loop : Indexer(... , (0,0), ...) , Indexer(... , (0,1), ...) , Indexer(... , (0,2), ...) , ...
# '_{a}^{b}' => return loop : Indexer(... , (0,0), ...) , Indexer(... , (0,1), ...) , Indexer(... , (1,0), ...) , Indexer(... , (1,1), ...) , ...
# '_{0}^{1}' => return Indexer(... , (0,1), ...)

# 'A_{c} = B_{c}*D_{c}'

# --- Scenario 1 ----
# 'G_{a}_{b} = G_{a}_{b} + G_{b}_{a}'

# --- Scenario 2 ----
# 'G_{a}_{b} = B_{a} + C_{b}'

#lis = list(it.product(np.arange(0, 4, 1), repeat = 4))
#[[i[:2],i[2:]] for i in lis if itemgetter(*[0,1])(i) == itemgetter(*[3,2])(i) and itemgetter(*[0,1])(i) == (1,0)]

Indexer(Metric, Metric, (1,0), [[0,1],[1,0]])

[[(1, 0), (0, 1)]]

In [ ]:
Gin = GrTensor(InvMetric, '^{a}^{b}')
G1 = GrDerivative(Base, '_{c}')*GrTensor(Metric, '_{d}_{b}')
G2 = GrDerivative(Base, '_{d}')*GrTensor(Metric, '_{c}_{b}')
G3 = GrDerivative(Base, '_{b}')*GrTensor(Metric, '_{c}_{d}')

In [ ]:
Gamma = (Gin/2)*(G1 + G2 - G3)
Gamma.components

[[[0, Derivative(A(r), r)/(2*A(r)), 0, 0], [Derivative(A(r), r)/(2*A(r)), 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[Derivative(A(r), r)/(2*B(r)), 0, 0, 0], [0, Derivative(B(r), r)/(2*B(r)), 0, 0], [0, 0, -r/B(r), 0], [0, 0, 0, -r*sin(theta)**2/B(r)]], [[0, 0, 0, 0], [0, 0, 1/r, 0], [0, 1/r, 0, 0], [0, 0, 0, -sin(2*theta)/2]], [[0, 0, 0, 0], [0, 0, 0, 1/r], [0, 0, 0, sin(2*theta)/(2*sin(theta)**2)], [0, 1/r, sin(2*theta)/(2*sin(theta)**2), 0]]]

In [ ]:
df = GrTensor(InvMetric, '^{a}^{b}')*(GrDerivative(Base, '_{c}')*GrTensor(Metric, '_{d}_{b}')) + GrTensor(InvMetric, '^{a}^{b}')*(GrDerivative(Base, '_{d}')*GrTensor(Metric, '_{c}_{b}')) - GrTensor(InvMetric, '^{a}^{b}')*(GrDerivative(Base, '_{b}')*GrTensor(Metric, '_{c}_{d}'))

In [ ]:
df.components/2

[[[0, Derivative(A(r), r)/(2*A(r)), 0, 0], [Derivative(A(r), r)/(2*A(r)), 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[Derivative(A(r), r)/(2*B(r)), 0, 0, 0], [0, Derivative(B(r), r)/(2*B(r)), 0, 0], [0, 0, -r/B(r), 0], [0, 0, 0, -r*sin(theta)**2/B(r)]], [[0, 0, 0, 0], [0, 0, 1/r, 0], [0, 1/r, 0, 0], [0, 0, 0, -sin(2*theta)/2]], [[0, 0, 0, 0], [0, 0, 0, 1/r], [0, 0, 0, sin(2*theta)/(2*sin(theta)**2)], [0, 1/r, sin(2*theta)/(2*sin(theta)**2), 0]]]

In [ ]:
BaseFunctions(Metric, Base).Gamma()

[[[0, Derivative(A(r), r)/(2*A(r)), 0, 0], [Derivative(A(r), r)/(2*A(r)), 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[Derivative(A(r), r)/(2*B(r)), 0, 0, 0], [0, Derivative(B(r), r)/(2*B(r)), 0, 0], [0, 0, -r/B(r), 0], [0, 0, 0, -r*sin(theta)**2/B(r)]], [[0, 0, 0, 0], [0, 0, 1/r, 0], [0, 1/r, 0, 0], [0, 0, 0, -sin(2*theta)/2]], [[0, 0, 0, 0], [0, 0, 0, 1/r], [0, 0, 0, 1/tan(theta)], [0, 1/r, 1/tan(theta), 0]]]